# Data Preprocessing

In [2]:
import math
import datetime
import numpy as np
import pandas as pd
import pygeohash as gh

## The Data

### Load the raw data

In [3]:
# !rm "traffic-management.zip" "training.csv"
# !wget "https://s3-ap-southeast-1.amazonaws.com/grab-aiforsea-dataset/traffic-management.zip"
# !unzip traffic-management.zip
# !mv "Traffic Management/training.csv" .
# !rm -r "Traffic Management"

In [4]:
df = pd.read_csv('training.csv')
df.head()

,geohash6,day,timestamp,demand
0,qp03wc,18,20:0,0.020072
1,qp03pn,10,14:30,0.024721
2,qp09sw,9,6:15,0.102821
3,qp0991,32,5:0,0.088755
4,qp090q,15,4:0,0.074468


### Feature Engineering

In [5]:
df['in_raw'] = True
df.head()

,geohash6,day,timestamp,demand,in_raw
0,qp03wc,18,20:0,0.020072,True
1,qp03pn,10,14:30,0.024721,True
2,qp09sw,9,6:15,0.102821,True
3,qp0991,32,5:0,0.088755,True
4,qp090q,15,4:0,0.074468,True


#### Add datetime features

In [6]:
df["weekday"] = df["day"].apply(lambda x : x % 7)

In [7]:
timestamps = df['timestamp'].unique()
timestamps_decoded = {}
for timestamp in timestamps:
    timestamps_decoded[timestamp] = list(map(int, timestamp.split(':')))

In [8]:
%%time
df['hour']            = df['timestamp'].apply(lambda x : timestamps_decoded[x][0])
df['minutes']         = df['timestamp'].apply(lambda x : timestamps_decoded[x][1])

Wall time: 6.59 s


In [9]:
%%time
df['ds'] = df['day'].astype(str) + " " + df['timestamp']
df['ds'] = df['ds'].apply(lambda x : datetime.datetime.strptime(x, "%j %H:%M"))

Wall time: 2min 12s


In [10]:
df.head()

,geohash6,day,timestamp,demand,in_raw,weekday,hour,minutes,ds
0,qp03wc,18,20:0,0.020072,True,4,20,0,1900-01-18 20:00:00
1,qp03pn,10,14:30,0.024721,True,3,14,30,1900-01-10 14:30:00
2,qp09sw,9,6:15,0.102821,True,2,6,15,1900-01-09 06:15:00
3,qp0991,32,5:0,0.088755,True,4,5,0,1900-02-01 05:00:00
4,qp090q,15,4:0,0.074468,True,1,4,0,1900-01-15 04:00:00


#### Get Unique locations

In [11]:
locations = df['geohash6'].unique()
print("The dataset has", len(locations), "unique geohashes:\n", locations)

The dataset has 1329 unique geohashes:
 ['qp03wc' 'qp03pn' 'qp09sw' ... 'qp03yn' 'qp09v9' 'qp0d45']


#### Fill in missing values

In [12]:
min_ds = df['ds'].min()
max_ds = df['ds'].max()
delta_ds = datetime.timedelta(minutes=15)
print(min_ds, '--to-->', max_ds, 'with steps:', delta_ds)

1900-01-01 00:00:00 --to--> 1900-03-02 23:45:00 with steps: 0:15:00


In [13]:
%%time
present_values = df[['geohash6', 'ds']].values
present_values = set(tuple(x) for x in present_values)

Wall time: 35.6 s


In [15]:
%%time
missing_values = []
for location in locations:
    cur_time = min_ds
    while cur_time <= max_ds:
        if not (location, cur_time) in present_values:
            day = cur_time.dayofyear
            x = {'geohash6' : location,
                 'day' : day,
                 'timestamp' : str(int(cur_time.hour)) + ":" + str(int(cur_time.minute)),
                 'demand' : 0,
                 'in_raw' : False,
                 'weekday' : day % 7,
                 'hour' : cur_time.hour,
                 'minutes' : cur_time.minute,
                 'ds' : cur_time}
            missing_values.append(x)
        cur_time += delta_ds

Wall time: 1min 54s


In [16]:
%%time
df_missing = pd.DataFrame(missing_values)

Wall time: 16.5 s


In [17]:
%%time
df_full = pd.concat([df, df_missing], sort=False)

Wall time: 2.4 s


In [18]:
df_full = df_full.reset_index().drop('index', axis=1)
df_full.head()

,geohash6,day,timestamp,demand,in_raw,weekday,hour,minutes,ds
0,qp03wc,18,20:0,0.020072,True,4,20,0,1900-01-18 20:00:00
1,qp03pn,10,14:30,0.024721,True,3,14,30,1900-01-10 14:30:00
2,qp09sw,9,6:15,0.102821,True,2,6,15,1900-01-09 06:15:00
3,qp0991,32,5:0,0.088755,True,4,5,0,1900-02-01 05:00:00
4,qp090q,15,4:0,0.074468,True,1,4,0,1900-01-15 04:00:00


In [19]:
len(df_full) == 4*24*61*len(locations)

True

#### Decode the geohashes

In [20]:
locations_decoded = {}
for location in locations:
    locations_decoded[location] = gh.decode_exactly(location)

In [21]:
%%time
df_full['latitude']  = df_full['geohash6'].apply(lambda x : locations_decoded[x][0])
df_full['longitude'] = df_full['geohash6'].apply(lambda x : locations_decoded[x][1])

Wall time: 16.2 s


In [22]:
df_full.head()

,geohash6,day,timestamp,demand,in_raw,weekday,hour,minutes,ds,latitude,longitude
0,qp03wc,18,20:0,0.020072,True,4,20,0,1900-01-18 20:00:00,-5.353088,90.653687
1,qp03pn,10,14:30,0.024721,True,3,14,30,1900-01-10 14:30:00,-5.413513,90.664673
2,qp09sw,9,6:15,0.102821,True,2,6,15,1900-01-09 06:15:00,-5.325623,90.906372
3,qp0991,32,5:0,0.088755,True,4,5,0,1900-02-01 05:00:00,-5.353088,90.752563
4,qp090q,15,4:0,0.074468,True,1,4,0,1900-01-15 04:00:00,-5.413513,90.719604


#### Circular Embedding

In [23]:
# %%time
# df_full['weekday_sin'] = np.sin(2*np.pi*df_full['weekday'].values/7)
# df_full['weekday_cos'] = np.cos(2*np.pi*df_full['weekday'].values/7)
# 
# df_full['hour_sin']    = np.sin(2*np.pi*df_full['hour'].values/24)
# df_full['hour_cos']    = np.cos(2*np.pi*df_full['hour'].values/24)
# 
# df_full['minutes_sin'] = np.sin(2*np.pi*df_full['minutes'].values/60)
# df_full['minutes_cos'] = np.cos(2*np.pi*df_full['minutes'].values/60)

In [24]:
# df_full.head()

### Sort by time

In [25]:
%%time
df_full = df_full.sort_values(by=['ds']).reset_index().drop('index', axis=1)

Wall time: 15.9 s


In [26]:
df_full.head()

,geohash6,day,timestamp,demand,in_raw,weekday,hour,minutes,ds,latitude,longitude
0,qp09du,1,0:0,0.000000,False,1,0,0,1900-01-01,-5.336609,90.829468
1,qp09jm,1,0:0,0.000000,False,1,0,0,1900-01-01,-5.419006,90.939331
2,qp09sf,1,0:0,0.000000,False,1,0,0,1900-01-01,-5.347595,90.917358
3,qp09vd,1,0:0,0.000000,False,1,0,0,1900-01-01,-5.303650,90.950317
4,qp03y4,1,0:0,0.002732,True,1,0,0,1900-01-01,-5.303650,90.620728


### Save preprocessed data

In [27]:
%%time
# to_drop = ['timestamp',
#            'weekday_sin', 'weekday_cos',
#            'hour_sin', 'hour_cos',
#            'minutes_sin', 'minutes_cos']
to_drop = ['timestamp']
df_full.drop(to_drop, axis=1).to_csv("full_data.csv", index=False)

Wall time: 6min 53s
